In [15]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from pathlib import Path
from calo_cluster.evaluation.utils import get_palette

In [26]:
root = Path('/global/cscratch1/sd/schuya/calo_cluster/plots/27qp2g8o/last')
nn_dir = root / 'meanshift_0.1'
pf_dir = root / 'pf'

In [27]:
nn_eff = np.load(str(nn_dir / '1' / 'efficiency.npy'))
nn_fr = np.load(str(nn_dir / '1' / 'fake_rate.npy'))
nn_response = np.load(str(nn_dir / '1' / 'response.npy'))
nn_resolution = np.load(str(nn_dir / '1' / 'resolution.npy'))

In [28]:
nn_resolution

array([0.17871329, 0.2676008 , 0.28023517, 0.27851146, 0.27126446,
       0.28589222, 0.3163366 ], dtype=float32)

In [29]:
from pathlib import Path
import sys
from typing import Union
import colorcet as cc
import numpy as np

def get_palette(instances: Union[np.array, int]):
    if type(instances) is int:
        size = instances
    else:
        size = np.unique(instances).shape[0]
    hex_colors = cc.glasbey_dark[:size]
    return [f'rgb{int(h[1:3], base=16),int(h[3:5], base=16),int(h[5:7], base=16)}' for h in hex_colors]

In [30]:
edges = np.array([0, 0.5, 1, 2, 3, 6, 10])
centers = (edges[1:] + edges[:-1]) / 2

In [31]:
def make_step_plot(edges, y_pf, y_nn):
    centers = (edges[1:] + edges[:-1]) / 2
    fig = go.Figure()
    colors = get_palette(2)
    fig.add_trace(go.Scatter(x=edges, y=y_pf, mode='lines', line_shape='hv', name='particle flow', legendgroup="particle flow", line_color=colors[0]))
    fig.add_trace(go.Scatter(x=centers, y=y_pf[:-1], mode='markers', name='particle flow', legendgroup="particle flow", line_color=colors[0]))
    fig.add_trace(go.Scatter(x=edges, y=y_nn, mode='lines', line_shape='hv', name='meanshift', legendgroup="meanshift", line_color=colors[1]))
    fig.add_trace(go.Scatter(x=centers, y=y_nn[:-1], mode='markers', name='meanshift', legendgroup="meanshift", line_color=colors[1]))
    return fig

In [32]:
def save_comparisons(root, pf_dir, nn_dir):
    pf_eff = np.load(str(pf_dir / '1' / 'efficiency.npy'))
    pf_fr = np.load(str(pf_dir / '1' / 'fake_rate.npy'))
    pf_response = np.load(str(pf_dir / '1' / 'response.npy'))
    pf_resolution = np.load(str(pf_dir / '1' / 'resolution.npy'))

    nn_eff = np.load(str(nn_dir / '1' / 'efficiency.npy'))
    nn_fr = np.load(str(nn_dir / '1' / 'fake_rate.npy'))
    nn_response = np.load(str(nn_dir / '1' / 'response.npy'))
    nn_resolution = np.load(str(nn_dir / '1' / 'resolution.npy'))
    
    eff_fig = make_step_plot(edges, pf_eff, nn_eff)
    eff_fig.update_layout(yaxis_range=(0,1), yaxis_title='efficiency', xaxis_title='energy [GeV]')
    eff_fig.show()
    eff_fig.write_image(str(root / 'efficiency.png'), scale=5)
    
    fr_fig = make_step_plot(edges, pf_fr, nn_fr)
    fr_fig.update_layout(yaxis_range=(0,1), yaxis_title='fake rate', xaxis_title='energy [GeV]')
    fr_fig.show()
    fr_fig.write_image(str(root / 'fake_rate.png'), scale=5)

    response_fig = make_step_plot(edges, pf_response, nn_response)
    response_fig.update_layout(yaxis_range=(0,2), yaxis_title='response', xaxis_title='energy [GeV]')
    response_fig.show()
    response_fig.write_image(str(root / 'response.png'), scale=5)

    resolution_fig = make_step_plot(edges, pf_resolution, nn_resolution)
    resolution_fig.update_layout(yaxis_range=(0,1), yaxis_title='resolution', xaxis_title='energy [GeV]')
    resolution_fig.show()
    resolution_fig.write_image(str(root / 'resolution.png'), scale=5)


In [33]:
save_comparisons(root, pf_dir, nn_dir)